# instance based Authorship attribution using n-gram

In [26]:
import numpy as np
from nltk import word_tokenize
from pip._internal.utils.misc import tabulate
from sklearn.feature_extraction.text import TfidfVectorizer
import glob
import pandas as pd
#%run stylometric_Features.ipynb import FeatureExtration
import seaborn as sns
import matplotlib as plt
import warnings
import string
from sklearn.pipeline import FeatureUnion
from sklearn.svm import SVC
from nltk.stem import WordNetLemmatizer
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import tree
import re
import nltk
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

In [27]:
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
all_instances={
'hrudayashiva':["01","02","03","04","05","06","07","08","09",10,11,12],
    'ravibeliger':[13,14,15,16,17,18,19,20,21,22,23,24,25],
    'somashaker':[26,27,28,29,30,31,32,33,34,35,36],
    'chandrashekark':[37,38,39,40,41,42,43,44,45,46,47],
    'usha':[48,49,50,51,52,53,54,55,56,57,58,59,60],
    'bhagavan':[61,62,63,64,65,66,67,68,69,70,71,72],
    'NaDisoza':[73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92],
    'srinatha':[93,94,95,96,97,98,99,100]
}


stop_words_list = "C://Users/RAVIKUMAR/PycharmProjects/Authorship_Attribution_Instance/Data_2/new_stop_words.txt"
data_folder = "C://Users/RAVIKUMAR/PycharmProjects/Authorship_Attribution_Instance/Data_2/train"
instance_by_author={}
for author,AllFiles_Per_author in all_instances.items():
    for i in AllFiles_Per_author:
        for name in glob.glob(f"C://Users/RAVIKUMAR/PycharmProjects/Authorship_Attribution_Instance/Data_2/train/instance{i}.txt"):
            with open(name, encoding='utf-8') as file:
                instance_by_author[author+str(i)]=file.read()

In [9]:
lemmatiser = WordNetLemmatizer()
stopword=[]

with open(stop_words_list, encoding='utf-8') as file:
    reader=file.read()
    reader = [reader.split()]
    stopword = sum(reader, [])

pp = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~0123456789'  
def text_process(text):
    nopunct = "".join([char for char in text if char not in pp])
    tokens = nltk.word_tokenize(nopunct) 
    nopunct = " ".join([word for word in tokens if word not in stopword])
    return nopunct


all_authors=[]
Clean_Data=[]
raw_data = []
for author, file in instance_by_author.items():
    all_authors.append(author)
    raw_data.append(file)
    Clean_Data.append(text_process(file))

In [30]:
df=pd.DataFrame()
df['author']=all_authors

Y = pd.Series(df["author"])
Y.replace(r'(^ra.*)','Ravibeligeri',regex=True, inplace = True)
Y.replace(r'(^hr.*)','Hrudayashiva',regex=True, inplace = True)
Y.replace(r'(^som.*)','Somashaker',regex=True, inplace = True)
Y.replace(r'(^ch.*)', 'Chandrashekar_k', regex=True, inplace=True)
Y.replace(r'(^us.*)', 'Usha', regex=True, inplace=True)
Y.replace(r'(^bh.*)', 'Bhagavan', regex=True, inplace=True)
Y.replace(r'(^Na.*)', 'NaDisoza', regex=True, inplace=True)
Y.replace(r'(^sri.*)', 'Srinatha', regex=True, inplace=True)
df['Clean_Data']=Clean_Data
df["raw_data"] = raw_data
y = Y
X = df['Clean_Data']

# 80-20 splitting the dataset (80%->Training and 20%->Validation)
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.35, random_state=6789)

In [11]:
def model_all(n_value_vector):  
    word_vector_train = n_value_vector.transform(X_train)
    word_vector_test = n_value_vector.transform(X_test)
    
    model1 =SVC(kernel='linear')
    model1 = model1.fit(word_vector_train, y_train)


    modelNB = GaussianNB()
    modelNB = modelNB.fit(word_vector_train.toarray(), y_train)

    model_Deci  = tree.DecisionTreeClassifier()
    model_Deci = model_Deci.fit(word_vector_train.toarray(), y_train)

    model_Rand = RandomForestClassifier(max_depth=2, random_state=0)
    model_Rand = model_Rand.fit(word_vector_train, y_train)
    
  
    print("SVM Training Accuracy        :",model1.score(word_vector_train, y_train))
    print("SVM Testing Acuuracy         : ",model1.score(word_vector_test, y_test))
    print()
    print("modelNB Training Accuracy    :",modelNB.score(word_vector_train.toarray(), y_train))
    print("modelNB Testing Acuuracy     : ",modelNB.score(word_vector_test.toarray(), y_test))
    print()
    print("model_Deci Training Accuracy :",model_Deci.score(word_vector_train.toarray(), y_train))
    print("model_Deci Testing Acuuracy  : ",model_Deci.score(word_vector_test.toarray(), y_test))
    print()
    print("model_Rand Training Accuracy :",model_Rand.score(word_vector_train, y_train))
    print("model_Rand Testing Acuuracy  : ",model_Rand.score(word_vector_test, y_test))
    return


# word and Char n-gram with n = 1,2,3,4,5,6

# n = 1

In [12]:
print("-------------word n gram----------- ")
one_word=TfidfVectorizer(analyzer="word", ngram_range=(1,1),max_features=2000, binary=False).fit(X_train)
model_all(one_word)
print("*************************************************************************************************")
print("---------------char n gram----------- ")
one_char=TfidfVectorizer(analyzer="char", ngram_range=(1,1),max_features=2000, binary=False).fit(X_train)
model_all(one_char)

-------------word n gram----------- 
SVM Training Accuracy        : 1.0
SVM Testing Acuuracy         :  0.8

modelNB Training Accuracy    : 1.0
modelNB Testing Acuuracy     :  0.5428571428571428

model_Deci Training Accuracy : 1.0
model_Deci Testing Acuuracy  :  0.6285714285714286

model_Rand Training Accuracy : 0.7384615384615385
model_Rand Testing Acuuracy  :  0.5428571428571428
*************************************************************************************************
---------------char n gram----------- 
SVM Training Accuracy        : 0.6
SVM Testing Acuuracy         :  0.6

modelNB Training Accuracy    : 0.9846153846153847
modelNB Testing Acuuracy     :  0.8571428571428571

model_Deci Training Accuracy : 1.0
model_Deci Testing Acuuracy  :  0.9428571428571428

model_Rand Training Accuracy : 0.7692307692307693
model_Rand Testing Acuuracy  :  0.6857142857142857


# n = 2

In [13]:
print("-------------word n gram----------- ")
two_word=TfidfVectorizer(analyzer="word", ngram_range=(2,2),max_features=2000, binary=False).fit(X_train)

model_all(two_word)
print("*************************************************************************************************")
print("---------------char n gram----------- ")
two_char=TfidfVectorizer(analyzer="char", ngram_range=(2,2),max_features=2000, binary=False).fit(X_train)
model_all(two_char)

-------------word n gram----------- 
SVM Training Accuracy        : 1.0
SVM Testing Acuuracy         :  0.8285714285714286

modelNB Training Accuracy    : 1.0
modelNB Testing Acuuracy     :  0.5142857142857142

model_Deci Training Accuracy : 1.0
model_Deci Testing Acuuracy  :  0.6285714285714286

model_Rand Training Accuracy : 0.6615384615384615
model_Rand Testing Acuuracy  :  0.4
*************************************************************************************************
---------------char n gram----------- 
SVM Training Accuracy        : 0.6923076923076923
SVM Testing Acuuracy         :  0.6

modelNB Training Accuracy    : 1.0
modelNB Testing Acuuracy     :  0.6285714285714286

model_Deci Training Accuracy : 1.0
model_Deci Testing Acuuracy  :  0.5714285714285714

model_Rand Training Accuracy : 0.8769230769230769
model_Rand Testing Acuuracy  :  0.7428571428571429


# n = 3

In [14]:
print("-------------word n gram----------- ")
three_word=TfidfVectorizer(analyzer="word", ngram_range=(3,3),max_features=2000, binary=False).fit(X_train)
model_all(three_word)
print("*************************************************************************************************")
print("---------------char n gram----------- ")
three_char=TfidfVectorizer(analyzer="char", ngram_range=(3,3),max_features=2000, binary=False).fit(X_train)
model_all(three_char)

-------------word n gram----------- 
SVM Training Accuracy        : 1.0
SVM Testing Acuuracy         :  0.6285714285714286

modelNB Training Accuracy    : 1.0
modelNB Testing Acuuracy     :  0.6857142857142857

model_Deci Training Accuracy : 1.0
model_Deci Testing Acuuracy  :  0.45714285714285713

model_Rand Training Accuracy : 0.5692307692307692
model_Rand Testing Acuuracy  :  0.3142857142857143
*************************************************************************************************
---------------char n gram----------- 
SVM Training Accuracy        : 0.9846153846153847
SVM Testing Acuuracy         :  0.8285714285714286

modelNB Training Accuracy    : 1.0
modelNB Testing Acuuracy     :  0.5428571428571428

model_Deci Training Accuracy : 1.0
model_Deci Testing Acuuracy  :  0.6

model_Rand Training Accuracy : 0.9230769230769231
model_Rand Testing Acuuracy  :  0.6857142857142857


# n = 4

In [15]:
print("-------------word n gram----------- ")
four_word=TfidfVectorizer(analyzer="word", ngram_range=(4,4),max_features=2000, binary=False).fit(X_train)
model_all(four_word)
print("*************************************************************************************************")
print("---------------char n gram----------- ")
four_char=TfidfVectorizer(analyzer="char", ngram_range=(4,4),max_features=2000, binary=False).fit(X_train)
model_all(four_char)

-------------word n gram----------- 
SVM Training Accuracy        : 0.9692307692307692
SVM Testing Acuuracy         :  0.3142857142857143

modelNB Training Accuracy    : 0.9846153846153847
modelNB Testing Acuuracy     :  0.37142857142857144

model_Deci Training Accuracy : 0.9846153846153847
model_Deci Testing Acuuracy  :  0.17142857142857143

model_Rand Training Accuracy : 0.49230769230769234
model_Rand Testing Acuuracy  :  0.11428571428571428
*************************************************************************************************
---------------char n gram----------- 
SVM Training Accuracy        : 1.0
SVM Testing Acuuracy         :  0.9142857142857143

modelNB Training Accuracy    : 1.0
modelNB Testing Acuuracy     :  0.5428571428571428

model_Deci Training Accuracy : 1.0
model_Deci Testing Acuuracy  :  0.6857142857142857

model_Rand Training Accuracy : 0.8461538461538461
model_Rand Testing Acuuracy  :  0.7142857142857143


# n = 5

In [16]:
print("-------------word n gram----------- ")
five_word=TfidfVectorizer(analyzer="word", ngram_range=(5,5),max_features=2000, binary=False).fit(X_train)
model_all(five_word)
print("*************************************************************************************************")
print("---------------char n gram----------- ")
five_char=TfidfVectorizer(analyzer="char", ngram_range=(5,5),max_features=2000, binary=False).fit(X_train)
model_all(five_char)

-------------word n gram----------- 
SVM Training Accuracy        : 0.9692307692307692
SVM Testing Acuuracy         :  0.14285714285714285

modelNB Training Accuracy    : 0.9692307692307692
modelNB Testing Acuuracy     :  0.14285714285714285

model_Deci Training Accuracy : 0.9846153846153847
model_Deci Testing Acuuracy  :  0.14285714285714285

model_Rand Training Accuracy : 0.4307692307692308
model_Rand Testing Acuuracy  :  0.14285714285714285
*************************************************************************************************
---------------char n gram----------- 
SVM Training Accuracy        : 1.0
SVM Testing Acuuracy         :  0.9428571428571428

modelNB Training Accuracy    : 1.0
modelNB Testing Acuuracy     :  0.5714285714285714

model_Deci Training Accuracy : 1.0
model_Deci Testing Acuuracy  :  0.8

model_Rand Training Accuracy : 0.8153846153846154
model_Rand Testing Acuuracy  :  0.6571428571428571


# n = 6

In [17]:
print("-------------word n gram----------- ")
six_word=TfidfVectorizer(analyzer="word", ngram_range=(6,6),max_features=2000, binary=False).fit(X_train)
model_all(six_word)
print("*************************************************************************************************")
print("---------------char n gram----------- ")
six_char=TfidfVectorizer(analyzer="char", ngram_range=(6,6),max_features=2000, binary=False).fit(X_train)
model_all(six_char)

-------------word n gram----------- 
SVM Training Accuracy        : 0.9538461538461539
SVM Testing Acuuracy         :  0.11428571428571428

modelNB Training Accuracy    : 0.9538461538461539
modelNB Testing Acuuracy     :  0.14285714285714285

model_Deci Training Accuracy : 0.9846153846153847
model_Deci Testing Acuuracy  :  0.11428571428571428

model_Rand Training Accuracy : 0.4153846153846154
model_Rand Testing Acuuracy  :  0.14285714285714285
*************************************************************************************************
---------------char n gram----------- 
SVM Training Accuracy        : 1.0
SVM Testing Acuuracy         :  0.9428571428571428

modelNB Training Accuracy    : 1.0
modelNB Testing Acuuracy     :  0.6

model_Deci Training Accuracy : 1.0
model_Deci Testing Acuuracy  :  0.8

model_Rand Training Accuracy : 0.8461538461538461
model_Rand Testing Acuuracy  :  0.8285714285714286


# Union

In [18]:
#calling each tf-idf vectors and union
UnionFeater = FeatureUnion([                        
                        ("word_1",one_word),("word_2",two_word),("word_3",three_word),("word_4",four_word),
                           ("word_5",five_word),
     ("char_1",one_char),("char_2",two_char),("char_3",three_char),("char_4",four_char),
                           ("char_5",five_char)
                           ] )

In [19]:
#TF-IDF vectorizer
vector_train = UnionFeater.transform(X_train)
vector_test = UnionFeater.transform(X_test)

SVM =SVC(kernel='linear')
SVM = SVM.fit(vector_train, y_train)

print("SVM Training Accuracy        :",SVM.score(vector_train, y_train))
print("SVM Testing Acuuracy         : ",SVM.score(vector_test, y_test))
predictions = SVM.predict(vector_test)
print(classification_report(y_test, predictions))

SVM Training Accuracy        : 1.0
SVM Testing Acuuracy         :  0.9428571428571428
                 precision    recall  f1-score   support

       Bhagavan       1.00      1.00      1.00         3
Chandrashekar_k       1.00      1.00      1.00         3
   Hrudayashiva       0.75      1.00      0.86         3
       NaDisoza       1.00      1.00      1.00        11
   Ravibeligeri       1.00      0.75      0.86         4
     Somashaker       1.00      0.75      0.86         4
       Srinatha       1.00      1.00      1.00         3
           Usha       0.80      1.00      0.89         4

    avg / total       0.96      0.94      0.94        35



In [20]:
cmodel_SVM = SVC(kernel='linear', C=1, random_state=42)
cmodel_NB = GaussianNB()
cmodel_Deci  = tree.DecisionTreeClassifier()
cmodel_Rand = RandomForestClassifier(max_depth=2, random_state=0)
cmodel = [ cmodel_NB,cmodel_Deci ,cmodel_Rand]

def allModel(model):
    vector_train = UnionFeater.transform(X_train)
    vector_test = UnionFeater.transform(X_test)
    model = model.fit(vector_train.toarray(), y_train)
    print(" Training Accuracy        :",model.score(vector_train.toarray(), y_train))
    print(" Testing Acuuracy         : ",model.score(vector_test.toarray(), y_test))
    predictions = model.predict(vector_test.toarray())
    print(classification_report(y_test, predictions))
for i in cmodel:
    print("Model used: ", i)
    allModel(i)

Model used:  GaussianNB(priors=None)
 Training Accuracy        : 1.0
 Testing Acuuracy         :  0.6571428571428571
                 precision    recall  f1-score   support

       Bhagavan       0.33      1.00      0.50         3
Chandrashekar_k       1.00      0.33      0.50         3
   Hrudayashiva       0.50      0.33      0.40         3
       NaDisoza       0.85      1.00      0.92        11
   Ravibeligeri       0.00      0.00      0.00         4
     Somashaker       0.00      0.00      0.00         4
       Srinatha       0.75      1.00      0.86         3
           Usha       0.67      1.00      0.80         4

    avg / total       0.56      0.66      0.57        35

Model used:  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=Fa

# cross Validation 

In [21]:
cross1=TfidfVectorizer(analyzer="word", ngram_range=(1,5),max_features=2000, binary=False).fit(X)
cross2=TfidfVectorizer(analyzer="char", ngram_range=(1,5),max_features=2000, binary=False).fit(X)
# cross_vecotr1 = cross1.transform(X)
# cross_vecotr2 = cross2.transform(X)



crossfeatures = FeatureUnion([
                        
                        ("word",cross1),("char",cross2)
                           
] )
cross_vecotr = crossfeatures.fit_transform(X)


authors = y


In [22]:
cmodel_SVM = SVC(kernel='linear', C=1, random_state=42)
cmodel_NB = GaussianNB()
cmodel_Deci  = tree.DecisionTreeClassifier()
cmodel_Rand = RandomForestClassifier(max_depth=2, random_state=0)
cmodel = [cmodel_NB,cmodel_Deci ,cmodel_Rand]


In [23]:
print("SVM cross_validation with CV = 10")
scores = cross_validate(cmodel_SVM, cross_vecotr, authors, cv=10)
print("train score: ", scores["train_score"])
print("test score: ", scores["test_score"])

SVM cross_validation with CV = 10
train score:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
test score:  [0.93333333 0.76923077 0.81818182 0.88888889 1.         0.88888889
 1.         1.         0.625      0.875     ]


In [24]:
for i in cmodel:
    scores = cross_validate(i, cross_vecotr.toarray(), authors, cv=10)
    print(i,scores)

GaussianNB(priors=None) {'fit_time': array([0.01799202, 0.0129962 , 0.01099849, 0.01099849, 0.01099539,
       0.01099658, 0.00999761, 0.01199985, 0.01099801, 0.01101804]), 'score_time': array([0.00801253, 0.00499821, 0.00399303, 0.00699377, 0.00399709,
       0.00400305, 0.0039978 , 0.00299501, 0.00299835, 0.00299764]), 'test_score': array([0.46666667, 0.46153846, 0.36363636, 0.33333333, 0.44444444,
       0.66666667, 0.44444444, 0.55555556, 0.25      , 0.5       ]), 'train_score': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best') {'fit_time': array([0.10297179, 0.11198449, 0.11195087, 0.11496091, 0.11494732,
       0.11596465, 0.11096025, 0.

# instance: each instance result -n gram with union

In [29]:
eachInstanceResult  = pd.DataFrame()
eachInstanceResult["instance"]=X_train
eachInstanceResult["vector:sparse matrix sum value"]=vector_train.sum(axis=1)
eachInstanceResult.head()

,instance,vector:sparse matrix sum value
72,ಕಾಡು ನಮ್ಮದಲ್ಲ ಒಳಹೋದಂತೆಲ್ಲ ದಟ್ಟವಾದ ಕಾಡು ಸೂರ್ಯನ ...,121.307195
83,ಮುದ್ದು ಹುಡುಗಿ ಮನೋಹರಿ ಹಾಗು ರಘುರಾಮರಿಗೆ ಹಿಂದಿನಂತಾ...,113.145777
21,ಅದಕ್ಕೆ ಯಾಕೆ ಹಾಗೆ ಹೆಸರಿಟ್ಟೆನೋ ಗೊತ್ತಿಲ್ಲ ಆಗಿನ್ನೂ...,103.622260
3,ಹೆಂಡತಿಯು ಹದ ಮಾಡಿಕೊಟ್ಟ ತಾಂಬೂಲ ಜಗಿಯುತ್ತಲೇ ರಾಜಾಜಿ...,109.012959
35,ಇರುವುದೆಲ್ಲವ ಬಿಟ್ಟು ಇರದುದರೆಡೆಗೆ ತುಡಿವುದೇ ಜೀವನ ಎ...,85.767055
